<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-библиотек-и-данных" data-toc-modified-id="Загрузка-библиотек-и-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек и данных</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение модели</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression-все-данные" data-toc-modified-id="LogisticRegression-все-данные-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>LogisticRegression все данные</a></span></li></ul></li></ul></div>

## Описание данных

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году

В колонках датасета содержатся следующие данные:
- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — median_house_value. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Загрузка библиотек и данных

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression


pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [2]:
RANDOM_SEED = 15364

In [3]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [4]:
df_housing = spark.read.load('./housing.csv', format="csv", sep=",", inferSchema=True, header="true")
df_housing.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

## Предобработка  

Проверим данные на пропуски.

In [5]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


В столбце `total_bedrooms` есть пропуски, заполнять их чем попало не получится, нужно просто удалить, учитывая что это всего 200 строк, примерно 1%

In [6]:
df_housing = df_housing.dropna(how = 'any', subset = 'total_bedrooms')

In [7]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20433,20433,20433,20433,20433,20433,20433,20433,20433,20433
1,mean,-119.57068859198068,35.63322125972706,28.633093525179856,2636.5042333480155,537.8705525375618,1424.9469485635982,499.43346547251997,3.8711616013312273,206864.41315519012,None
2,stddev,2.003577890751096,2.1363476663779872,12.591805202182835,2185.269566977601,421.38507007403115,1133.2084897449597,382.2992258828481,1.899291249306247,115435.66709858322,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Теперь можно перевести категориальные значения в понятные для модели цифры через OHE.

In [8]:
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx']) 
df_housing = indexer.fit(df_housing).transform(df_housing)

encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe'])
df_housing = encoder.fit(df_housing).transform(df_housing)

categorical_assembler =VectorAssembler(inputCols=['ocean_proximity_ohe'],
                                        outputCol="categorical_features")
df_housing = categorical_assembler.transform(df_housing)

In [9]:
df_housing.toPandas().sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,categorical_features
14862,-117.03,32.77,34.0,1796.0,428.0,918.0,424.0,2.8750,161200.0,<1H OCEAN,0.0,[0.0]
2201,-119.85,36.84,12.0,2272.0,304.0,840.0,305.0,8.9669,213900.0,INLAND,1.0,[1.0]
15565,-122.43,37.79,24.0,2459.0,1001.0,1362.0,957.0,2.6782,450000.0,NEAR BAY,3.0,[3.0]
19210,-120.93,37.74,37.0,1956.0,402.0,1265.0,397.0,2.3023,91900.0,INLAND,1.0,[1.0]
8894,-118.44,33.98,21.0,18132.0,5419.0,7431.0,4930.0,5.3359,500001.0,<1H OCEAN,0.0,[0.0]


Теперь можно и числовые столбцы привести к одному размеру.

In [10]:
numerical_assembler = VectorAssembler(inputCols=['longitude',
                                                 'latitude',
                                                 'housing_median_age',
                                                 'total_rooms',
                                                 'total_bedrooms',
                                                 'population',
                                                 'households',
                                                 'median_income',],
                                      outputCol="numerical_features")
df_housing = numerical_assembler.transform(df_housing) 
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled")

df_housing = standardScaler.fit(df_housing).transform(df_housing) 

In [11]:
df_housing.toPandas().sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,categorical_features,numerical_features,numerical_features_scaled
823,-122.08,37.61,26.0,2261.0,443.0,1039.0,395.0,3.7931,203900.0,NEAR BAY,3.0,[3.0],"[-122.08, 37.61, 26.0, 2261.0, 443.0, 1039.0, ...","[-60.93099777330593, 17.604812452537214, 2.064..."
5137,-118.26,33.93,36.0,1102.0,247.0,702.0,225.0,1.5256,95400.0,<1H OCEAN,0.0,[0.0],"[-118.26, 33.93, 36.0, 1102.0, 247.0, 702.0, 2...","[-59.02440855726704, 15.882246384328308, 2.859..."
14197,-117.16,32.72,27.0,1245.0,471.0,653.0,451.0,1.2668,225000.0,NEAR OCEAN,2.0,[2.0],"[-117.16, 32.72, 27.0, 1245.0, 471.0, 653.0, 4...","[-58.47539072018777, 15.315859171683533, 2.144..."
10124,-117.88,33.87,35.0,1919.0,349.0,1302.0,345.0,5.6409,190900.0,<1H OCEAN,0.0,[0.0],"[-117.88, 33.87, 35.0, 1919.0, 349.0, 1302.0, ...","[-58.834747849912375, 15.854161067998815, 2.77..."
5359,-118.44,34.02,32.0,2242.0,490.0,921.0,461.0,4.0429,500001.0,<1H OCEAN,0.0,[0.0],"[-118.44, 34.02, 32.0, 2242.0, 490.0, 921.0, 4...","[-59.11424783969819, 15.92437435882255, 2.5413..."


Сибираем все вместе для будущего обучения.

In [12]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol='features') 
df_housing = final_assembler.transform(df_housing)

In [13]:
df_housing.select(['features', 'median_house_value']).toPandas().head(5)

,features,median_house_value
0,"[3.0, -61.00586384199856, 17.731196376019934, ...",452600.0
1,"[3.0, -61.00087277075238, 17.721834603910104, ...",358500.0
2,"[3.0, -61.010854913244735, 17.717153717855187,...",352100.0
3,"[3.0, -61.01584598449091, 17.717153717855187, ...",341300.0
4,"[3.0, -61.01584598449091, 17.717153717855187, ...",342200.0


Теперь делим на обучающую и проверочную выборки.

In [14]:
train_data, test_data = df_housing.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16225 4208


In [15]:
df_housing.select('features','median_house_value').show(5)

+--------------------+------------------+
|            features|median_house_value|
+--------------------+------------------+
|[3.0,-61.00586384...|          452600.0|
|[3.0,-61.00087277...|          358500.0|
|[3.0,-61.01085491...|          352100.0|
|[3.0,-61.01584598...|          341300.0|
|[3.0,-61.01584598...|          342200.0|
+--------------------+------------------+
only showing top 5 rows



In [16]:
train_data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|categorical_features|  numerical_features|numerical_features_scaled|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+--------------------+--------------------+-------------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|               [2.0]|[-124.35,40.54,52...|     [-62.063970946187...|[2.0,-62.06397094...|
|   -124.3| 

Отлично, все готово для будущего обучения.

## Обучение модели

Для обучения модели будем использовать несолько подходов:
1. Обучать нразные модели
2. Использовать разное количество данных.

### LogisticRegression все данные

In [19]:
lr = LogisticRegression(labelCol='median_house_value', featuresCol='numerical_features_scaled',maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [20]:
model = lr.fit(train_data) 

Py4JJavaError: An error occurred while calling o258.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 23) (DESKTOP-UD3OFTF executor driver): java.lang.IllegalArgumentException: requirement failed: 14563 x 500002 dense matrix is too large to allocate
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.linalg.DenseMatrix$.zeros(Matrices.scala:490)
	at org.apache.spark.ml.optim.aggregator.MultinomialLogisticBlockAggregator.add(MultinomialLogisticBlockAggregator.scala:112)
	at org.apache.spark.ml.optim.aggregator.MultinomialLogisticBlockAggregator.add(MultinomialLogisticBlockAggregator.scala:45)
	at org.apache.spark.ml.optim.loss.RDDLossFunction.$anonfun$calculate$1(RDDLossFunction.scala:59)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:194)
	at sc

In [ ]:
predictions = model.transform(test_data)
predictions.select("Survived", "prediction").show(10) 